In [ ]:
import functools
import pathlib

import lifetimes

In [ ]:
lopath = pathlib.Path("/home/fabian/Documents/MAELSTROM/data/pca/")
outfile = f"{local_path}/temperature_level_128_daily_averages_2017_2020.nc"
paths = lifetimes.utils.list_files(path, pattern="ml*.nc")
paths

In [ ]:
preprocess = functools.partial(
    lifetimes.features.methods.select_model_level_and_calculate_daily_mean,
    level=128,
)
ds = lifetimes.features.EcmwfIfsHresDataset(
    paths=paths,
    overlapping=True,
    preprocessing=preprocess,
)

In [ ]:
ds.to_netcdf(outfile, drop_variables=["q", "u", "v"])